# **AlphaProteo Target Protein Specification**

We recognize that for non-structural biologists, accurately defining the target region of a protein can be challenging, particularly due to the confusion introduced by AUTH numbering ([see FAQ](https://docs.google.com/document/d/1wYLQve9gp59UL_8jFKrdZoZC7kS9SfimTEw5SIwjJP8/preview?tab=t.0#heading=h.3gm45nxykaln) for more details). This Colab aims to provide an interface for specifying target protein regions, performing basic validation, and visualising the selected target.

To begin, please type or paste your target PDB ID into the corresponding field of Cell 3, then select "Runtime" -> "Run all" from the menu. In Cell 4, use the drop-down lists to select your desired target residue ranges and hotspots. Buttons are available to add more residue ranges or hotspots as needed. To visualise your selection and hotspots on the structure, click the "Display Specification" button.

Once satisfied with your selection, please copy the PDB ID (from Cell 3), along with the Target Protein Specification and Hotspots (if any) from Cell 4, into the "Submit Design Request" form. Additionally, use the maximal binder size information provided in Cell 4 to populate the "Desired binder length range" section of the form.


In [ ]:
#@title 1. Install dependencies

print(f"Installing biopython and 3dmol.js... ")
!pip install -q --no-warn-conflicts py3Dmol biopython

import os
import re
import string
import urllib.request

import py3Dmol
from Bio.PDB import MMCIFParser
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

from IPython.display import display, update_display
import ipywidgets as widgets

In [ ]:
#@title 2. Helper functions

def split_hs_string(s):
  """Splits a string like 'A10' or 'B20' into two variables.

  Args:
    s: The string to split.

  Returns:
    A tuple containing the initial letter(s) and the number, or None if the
    string doesn't match the expected pattern.
  """
  match = re.match(r'([A-Za-z]+)(\d+)', s)
  if match:
    letters = match.group(1)
    number = int(match.group(2))
    return letters, number
  else:
    return None


def split_spec_string(s):
  """Splits a string like 'A10-15' or 'B20' into two variables.

  Args:
    s: The string to split.

  Returns:
  """
  match = re.match(r'([A-Za-z]+)([0-9-]+)', s)
  if match:
    letters = match.group(1)
    number = match.group(2)
    return letters, number
  else:
    return None


def parse_hotspot(pdb_id, hotspot):
  """Parses a hotspot string and returns a list of (chain, residue) tuples."""
  hs_parsed = []
  hs_res_list=hotspot.split(sep=',')
  for hs_residue in hs_res_list:
    hs = split_hs_string(hs_residue)
    if hs:
      hs_parsed.append(hs)
    else:
      raise RuntimeError(f"Hotspot definition '{hs_residue}' seems not to match the format")
  return hs_parsed


def parse_spec(pdb_id, spec):
  """Parses a specification string and returns a list of (chain, residue) tuples."""
  spec_parsed = []
  spec_chain_list=spec.split(sep='/')
  for chain in spec_chain_list:
    spec_list=chain.split(sep=',')
    for sp_residue in spec_list:
      sp = split_spec_string(sp_residue)
      if sp:
        spec_parsed.append(sp)
      else:
        raise RuntimeError(f"Protein specification '{sp_residue}' seems not to match the format")
  return spec_parsed


def get_structure(pdb_id):
    """
    Downloads an mmCIF file from the PDB and parses it

    Args:
        pdb_id: The PDB ID of the protein structure.
    Returns:
        structure by MMCIFParser
    """

    url = f"https://files.wwpdb.org/pub/pdb/data/structures/all/mmCIF/{pdb_id}.cif.gz"
    try:
        urllib.request.urlretrieve(url, f"{pdb_id}.cif.gz")
        os.system(f"gunzip {pdb_id}.cif.gz")
    except Exception as e:
        print(f"Error downloading file: {e}")
        return

    return f"{pdb_id}.cif"


def get_chain_residue_ids_from_mmcif(mmcif_file_path):
    """Parses mmCIF file and returns a dictionary of chain IDs to residue IDs."""

    # Load the mmCIF file using MMCIF2Dict
    mmcif_dict = MMCIF2Dict(mmcif_file_path)

    # Extract relevant columns
    label_asym_ids = mmcif_dict["_atom_site.label_asym_id"]
    auth_asym_ids = mmcif_dict["_atom_site.auth_asym_id"]
    label_seq_ids = mmcif_dict["_atom_site.label_seq_id"]
    auth_seq_ids = mmcif_dict["_atom_site.auth_seq_id"]
    label_comp_ids = mmcif_dict["_atom_site.label_comp_id"] # Extract residue names


    # Create a dictionary to store the mapping
    residue_mapping = {}
    for label_asym, auth_asym, label_seq, auth_seq, label_comp_id in zip(label_asym_ids, auth_asym_ids, label_seq_ids, auth_seq_ids, label_comp_ids):
        if label_comp_id in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']:
            key = (label_asym, label_seq)  # (Chain, Residue Number)
            value = (auth_asym, auth_seq)  # (Chain, Residue Number in auth numbering)
            residue_mapping[key] = value

    # Create the chain_residue_dict using the mapping
    chain_residue_dict = {}
    for (label_chain, label_resid), (auth_chain, auth_resid) in residue_mapping.items():
        if label_chain not in chain_residue_dict:
            chain_residue_dict[label_chain] = []
        # chain_residue_dict[label_chain].append((label_resid, auth_resid))  # Store both numberings as a tuple
        chain_residue_dict[label_chain].append(label_resid)  # Store both numberings as a tuple

    for chain_id in chain_residue_dict:
        seen = set()
        chain_residue_dict[chain_id] = [a for a in chain_residue_dict[chain_id] if not (a in seen or seen.add(a))]


    return chain_residue_dict


# --- Functions to Create Molecuar Graphics ---

def display_structure(pdb_id, spec, hotspot, use_auth, mol_widget):
  """Displays a protein structure using 3Dmol.js."""

  with mol_widget:
    viewer = py3Dmol.view(query=f'pdb:{pdb_id}', width=600, height=600)
    viewer.setStyle({'cartoon': {'colorscheme': 'chain'}})

    # Display user's selection
    if len(spec) > 0:
      spec_parsed = parse_spec(pdb_id, spec)
      viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'opacity': 0.6}})
      for sp in spec_parsed:
        if use_auth:
          viewer.setStyle({'chain': sp[0],'resi': sp[1]},{'cartoon': {'colorscheme': 'chain'}})
        else:
          viewer.setStyle({'lchain': sp[0],'lresi': sp[1]},{'cartoon': {'colorscheme': 'chain'}})


    # Display hotspots
    if len(hotspot) > 0:
      hs_parsed = parse_hotspot(pdb_id, hotspot)
      for hs in hs_parsed:
        if use_auth:
          viewer.setStyle({'chain': hs[0],'resi': hs[1]},{'sphere': {'colorscheme': 'whiteCarbon'}})
        else:
          viewer.setStyle({'lchain': hs[0],'lresi': hs[1]},{'sphere': {'colorscheme': 'whiteCarbon'}})

    viewer.setHoverable({},True,'''function(atom,viewer,event,container) {
                    if(!atom.label) {
                      atom.label = viewer.addLabel(atom.lchain+atom.lresi+"/"+atom.lresn,{position: atom, backgroundColor: 'mintcream', fontColor:'black'});

                    }}''',
                '''function(atom,viewer) {
                    if(atom.label) {
                      viewer.removeLabel(atom.label);
                      delete atom.label;
                    }
                  }''')

    viewer.zoomTo()
    viewer.show()
    return viewer


def update_structure(viewer, spec, hotspot, use_auth, mol_widget):
  """Updates a protein structure using 3Dmol.js."""

  with mol_widget:
    mol_widget.clear_output()

    if not viewer:
      print('ERROR: No viewer provided!')
      return

    viewer.setStyle({'cartoon': {'colorscheme': 'chain'}})

    # Display user's selection
    if len(spec) > 0:
      spec_parsed = parse_spec(pdb_id, spec)
      viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'opacity': 0.6}})
      for sp in spec_parsed:
        if use_auth:
          viewer.setStyle({'chain': sp[0],'resi': sp[1]},{'cartoon': {'colorscheme': 'chain'}})
        else:
          viewer.setStyle({'lchain': sp[0],'lresi': sp[1]},{'cartoon': {'colorscheme': 'chain'}})


    # Display hotspots
    if len(hotspot) > 0:
      hs_parsed = parse_hotspot(pdb_id, hotspot)
      for hs in hs_parsed:
        if use_auth:
          viewer.setStyle({'chain': hs[0],'resi': hs[1]},{'sphere': {'colorscheme': 'whiteCarbon'}})
        else:
          viewer.setStyle({'lchain': hs[0],'lresi': hs[1]},{'sphere': {'colorscheme': 'whiteCarbon'}})

    viewer.setHoverable({},True,'''function(atom,viewer,event,container) {
                    if(!atom.label) {
                      atom.label = viewer.addLabel(atom.lchain+atom.lresi+"/"+atom.lresn,{position: atom, backgroundColor: 'mintcream', fontColor:'black'});
                    }}''',
                '''function(atom,viewer) {
                    if(atom.label) {
                      viewer.removeLabel(atom.label);
                      delete atom.label;
                    }
                  }''')

    viewer.zoomTo()
    viewer.show()
    return viewer


In [ ]:
#@title 3. Get PDB structure

pdb_id = '5vli' #@param {type:"string"}
#@markdown - e.g. 5vli

print(f'Downloading {pdb_id}...')
mmcif_file = get_structure(pdb_id.lower())

try:
    chain_residue_data = get_chain_residue_ids_from_mmcif(mmcif_file)
    available_chains = sorted(chain_residue_data.keys()) # Get chains from dictionary keys
    print('Done!')
except FileNotFoundError:
    print(f"Error: mmCIF file not found at '{mmcif_file}'. Please upload or specify the correct file path.")
    chain_residue_data = {}
    available_chains = []


In [ ]:
#@title 4. Visual selection of the target protein specification and hotspots

# --- 3. Initialize Widget Lists and Global Variables ---
range_chain_dropdowns = []  # List to store dropdown widgets for chain selection in range selectors
range_residue_dropdowns_start = []  # List to store dropdown widgets for start residue selection in range selectors
range_residue_dropdowns_end = []  # List to store dropdown widgets for end residue selection in range selectors

hotspot_chain_dropdowns = []  # List to store dropdown widgets for chain selection in hotspot selectors
hotspot_residue_dropdowns = []  # List to store dropdown widgets for residue selection in hotspot selectors

target_protein_specification = ''  # String to store the protein specification based on user selections
hotspot_specification = '' # String to store hotspot specification


selection_widgets_ranges = None  # VBox to hold range selection widgets
selection_widgets_hotspots = None  # VBox to hold hotspot selection widgets

binder_length = 0

# --- 4. Functions to Create Interactive Dropdown Selectors ---

def create_chain_residue_range_dropdowns():
    """
    Creates a new set of dropdown widgets for selecting a chain and a residue range.
    Appends these widgets to the global lists and displays them in the Colab output.
    """
    global selection_widgets_ranges  # Access the global VBox for range selectors

    if available_chains:  # Only create dropdowns if chain data is available
        # --- Create widgets for chain selection ---
        chain_label = widgets.Label(value=f'Selection {len(range_chain_dropdowns) + 1}: chain ')
        chain_dropdown = widgets.Dropdown(
            options=available_chains,
            disabled=False,
            layout=widgets.Layout(width='10%'),  # Adjust width for layout
        )

        # --- Create widgets for residue range start selection ---
        residue_options = chain_residue_data.get(available_chains[0], [])  # Initial residue options based on the first chain
        residue_label_start = widgets.Label(value=f', residue range from ')
        residue_dropdown_start = widgets.Dropdown(
            options = [str(res_id) for res_id in residue_options],  # Convert residue IDs to strings for dropdown
            disabled=False,
            layout=widgets.Layout(width='10%'),
        )

        # --- Create widgets for residue range end selection ---
        residue_label_end = widgets.Label(value=f' to ')
        residue_dropdown_end = widgets.Dropdown(
            options = [str(res_id) for res_id in residue_options][1:],  # Options for end residue start from the second residue
            disabled=False,
            layout=widgets.Layout(width='10%'),
        )


        def chain_dropdown_callback(change):
            """
            Callback function: Updates residue dropdown options when the chain selection changes.
            """
            selected_chain = change.new  # Get the newly selected chain ID
            if selected_chain:
                residue_options = chain_residue_data.get(selected_chain, [])  # Get residue IDs for the selected chain
                residue_dropdown_start.options = [str(res_id) for res_id in residue_options]  # Update start residue dropdown
                residue_dropdown_end.options = [str(res_id) for res_id in residue_options][1:]  # Update end residue dropdown, starting from second residue
            else:
                residue_dropdown_start.options = []  # Clear residue dropdowns if no chain is selected
                residue_dropdown_end.options = []
            collect_selections_clicked('')


        def first_residue_dropdown_callback(change):
            """
            Callback function: Updates the end residue dropdown options based on the selected start residue.
            """
            selected_residue_start = change.new  # Get the newly selected start residue
            if selected_residue_start:
                residue_options =  [int(res_id) for res_id in residue_dropdown_start.options]  # Get residue options as integers
                residue_dropdown_end.options = [str(res_id) for res_id in residue_options if res_id > int(selected_residue_start)]  # Update end residue dropdown with residues after the start residue
            else:
                residue_dropdown_end.options = []  # Clear end residue dropdown if no start residue is selected
            collect_selections_clicked('')


        def second_residue_dropdown_callback(change):
            """
            Callback function: Updates target specification.
            """
            collect_selections_clicked('')



        chain_dropdown.observe(chain_dropdown_callback, names='value')  # Attach callback to chain dropdown
        residue_dropdown_start.observe(first_residue_dropdown_callback, names='value')  # Attach callback to start residue dropdown
        residue_dropdown_end.observe(second_residue_dropdown_callback, names='value')  # Attach callback to start residue dropdown

        range_chain_dropdowns.append(chain_dropdown)  # Add dropdowns to their respective lists
        range_residue_dropdowns_start.append(residue_dropdown_start)
        range_residue_dropdowns_end.append(residue_dropdown_end)

        # Display widgets in a horizontal layout within a vertical box
        if not selection_widgets_ranges:
            selection_widgets_ranges = widgets.VBox([widgets.HBox([chain_label, chain_dropdown, residue_label_start, residue_dropdown_start, residue_label_end, residue_dropdown_end])])
            display(selection_widgets_ranges)  # Display the VBox if it's the first selection
        else:
            selection_widgets_ranges.children += (widgets.HBox([chain_label, chain_dropdown, residue_label_start, residue_dropdown_start, residue_label_end, residue_dropdown_end]),)  # Append new HBox to existing VBox
        collect_selections_clicked('')
    else:
        print("No chains available to select.")



def create_hotspot_dropdowns():
    """
    Creates a new set of dropdown widgets for selecting a chain and a single hotspot residue.
    Appends these widgets to the global lists and displays them in the Colab output.
    """
    global selection_widgets_hotspots  # Access the global VBox for hotspot selectors

    if available_chains:  # Only create dropdowns if chain data is available
        # --- Create widgets for hotspot chain selection ---
        chain_label = widgets.Label(value=f'Hotspot {len(hotspot_chain_dropdowns) + 1}: chain ')
        hotspot_chain_dropdown = widgets.Dropdown(
            options=available_chains,
            disabled=False,
            layout=widgets.Layout(width='10%'),  # Adjust width for layout
        )

        # --- Create widgets for hotspot residue selection ---
        residue_options = chain_residue_data.get(available_chains[0], [])  # Initial residue options based on the first chain
        residue_label = widgets.Label(value=f', residue ')
        hotspot_residue_dropdown = widgets.Dropdown(
            options = [str(res_id) for res_id in residue_options],  # Convert residue IDs to strings for dropdown
            disabled=False,
            layout=widgets.Layout(width='10%'),
        )

        def hotspot_chain_dropdown_callback(change):
            """
            Callback function: Updates hotspot residue dropdown options when the chain selection changes.
            """
            selected_chain = change.new  # Get the newly selected chain ID
            if selected_chain:
                residue_options = chain_residue_data.get(selected_chain, [])  # Get residue IDs for the selected chain
                hotspot_residue_dropdown.options = [str(res_id) for res_id in residue_options]  # Update residue dropdown
            else:
                hotspot_residue_dropdown.options = []  # Clear residue dropdown if no chain is selected
            collect_selections_clicked('')

        def hotspot_residue_dropdown_callback(change):
            """
            Callback function: Updates specification.
            """
            collect_selections_clicked('')

        hotspot_chain_dropdown.observe(hotspot_chain_dropdown_callback, names='value')  # Attach callback to chain dropdown
        hotspot_residue_dropdown.observe(hotspot_residue_dropdown_callback, names='value')  # Attach callback to residue dropdown

        hotspot_chain_dropdowns.append(hotspot_chain_dropdown)  # Add dropdowns to their respective lists
        hotspot_residue_dropdowns.append(hotspot_residue_dropdown)


        # Display widgets in a horizontal layout within a vertical box
        if not selection_widgets_hotspots:
            selection_widgets_hotspots = widgets.VBox([widgets.HBox([chain_label, hotspot_chain_dropdown, residue_label, hotspot_residue_dropdown])])
            display(selection_widgets_hotspots)  # Display the VBox if it's the first hotspot selector
        else:
            selection_widgets_hotspots.children += (widgets.HBox([chain_label, hotspot_chain_dropdown, residue_label, hotspot_residue_dropdown]),)  # Append new HBox to existing VBox

        collect_selections_clicked('')

    else:
        print("No chains available to select.")



# --- 5. Function to Collect User Selections and Update Specification Text Widgets ---

def display_selections_clicked(b):
    """
    Callback function for the 'Display Selection' button.
    Updates molecular viewer
    """
    update_structure(mol_viewer, text_widget.value, hotspot_text_widget.value, False, mol_widget)


def collect_selections_clicked(b):
    """
    Collects selected chain and residue ranges/hotspots from all dropdowns and updates the specification text widgets.
    """
    global target_protein_specification  # Access the global specification string variable for ranges
    global hotspot_specification # Access the global specification string variable for hotspots

    target_protein_specification = ''  # Reset the specification string for range selections for new update
    hotspot_specification = ''  # Reset the hotspot specification string for new update


    used_chains_ranges = []  # Keep track of chains used in range selections for consecutive chain checking
    hotspots = []  # List to store hotspot selections (chain and residue)
    specification_length = 0


    for i in range(len(range_chain_dropdowns)):  # Iterate through range selection dropdowns
        chain_dropdown = range_chain_dropdowns[i]
        residue_dropdown_start = range_residue_dropdowns_start[i]
        residue_dropdown_end = range_residue_dropdowns_end[i]
        selected_chain = chain_dropdown.value  # Get selected chain ID
        selected_residue_segment_start = residue_dropdown_start.value  # Get selected start residue
        selected_residue_segment_end = residue_dropdown_end.value  # Get selected end residue

        # Check if chain and residue range are selected
        if selected_chain and selected_residue_segment_start and selected_residue_segment_end:

            # Build range specification string
            if target_protein_specification == '':  # If it's the first selection
                target_protein_specification = f'{selected_chain}{selected_residue_segment_start}-{selected_residue_segment_end}'
                used_chains_ranges.append(selected_chain)  # Add chain to used chains list
                specification_length = int(selected_residue_segment_end) - int(selected_residue_segment_start) + 1
            else:
                if selected_chain not in used_chains_ranges:  # If chain is new, add chain separator '/'
                    target_protein_specification += '/'
                    used_chains_ranges.append(selected_chain)
                elif selected_chain == used_chains_ranges[-1]:  # If chain is the same as the last one, add residue range separator ','
                    target_protein_specification += ','
                else:  # Error if chains are not selected consecutively
                    with output_area:
                        output_area.clear_output()  # Clear output area for error message
                        print(f'ERROR: Selection {i+1} - make sure chains are selected consecutively (i.e. A,A,B,B rather than A,B,A,B)')
                        return  # Exit the function if there's an error

                target_protein_specification += f'{selected_chain}{selected_residue_segment_start}-{selected_residue_segment_end}'  # Append range specification
                specification_length += int(selected_residue_segment_end) - int(selected_residue_segment_start) + 1


    for i in range(len(hotspot_chain_dropdowns)):  # Iterate through hotspot selection dropdowns
        hotspot_chain_dropdown = hotspot_chain_dropdowns[i]
        hotspot_residue_dropdown = hotspot_residue_dropdowns[i]
        selected_hotspot_chain = hotspot_chain_dropdown.value  # Get selected hotspot chain
        selected_hotspot_residue = hotspot_residue_dropdown.value  # Get selected hotspot residue

        # Check if both chain and residue are selected for hotspot
        if selected_hotspot_chain and selected_hotspot_residue:
            if hotspot_specification == '':  # If it's the first hotspot selection
                hotspot_specification = f'{selected_hotspot_chain}{selected_hotspot_residue}'
            else:
                hotspot_specification += f',{selected_hotspot_chain}{selected_hotspot_residue}'  # Append hotspot specification

    text_widget.value = target_protein_specification  # Update the text widget for range specification
    hotspot_text_widget.value = hotspot_specification # Update the text widget for hotspot specification
    binder_length = 512-specification_length
    with output_area:
        output_area.clear_output()  # Clear output area for error message
        print(f'You have specified {specification_length} residues for the target; this leaves up to {binder_length} residues for the binder')
        if binder_length < 60:
          print(f'WARNING: {binder_length} residues left for the binder is too small; please choose smaller target protein')


# --- 6. Button Click Handlers ---
def add_range_selection_clicked(b):
    """Callback function for the 'Add Target Selection' button."""
    create_chain_residue_range_dropdowns()  # Call function to create new range selection dropdowns

def add_hotspot_selection_clicked(b):
    """Callback function for the 'Add Hotspot Selection' button."""
    create_hotspot_dropdowns()  # Call function to create new hotspot selection dropdowns

def remove_range_selection_clicked(b):
    """Callback function for the 'Remove Target Selection' button."""
    global selection_widgets_ranges, range_chain_dropdowns, range_residue_dropdowns_start, range_residue_dropdowns_end
    if selection_widgets_ranges.children:  # Check if there are children to remove
        # Remove the last HBox (containing the range selection widgets)
        selection_widgets_ranges.children = selection_widgets_ranges.children[:-1]
        # Remove corresponding entries from the dropdown lists
        range_chain_dropdowns.pop()
        range_residue_dropdowns_start.pop()
        range_residue_dropdowns_end.pop()
        collect_selections_clicked('')  # Update the specification text widget

def remove_hotspot_selection_clicked(b):
    """Callback function for the 'Remove Hotspot' button."""
    global selection_widgets_hotspots, hotspot_chain_dropdowns, hotspot_residue_dropdowns
    if selection_widgets_hotspots:
        if selection_widgets_hotspots.children:  # Check if there are children to remove
            # Remove the last HBox (containing the hotspot selection widgets)
            selection_widgets_hotspots.children = selection_widgets_hotspots.children[:-1]
            # Remove corresponding entries from the dropdown lists
            hotspot_chain_dropdowns.pop()
            hotspot_residue_dropdowns.pop()
            collect_selections_clicked('')  # Update the specification text widget

def add_separator(thin=False, transparent = False):
    """Displaying visual separator"""
    if thin:
        if transparent:
            separator = widgets.HTML('<hr style="border: 1px solid transparent; margin: 5px 0;">')
        else:
            separator = widgets.HTML('<hr style="border: 1px solid #ccc; margin: 5px 0;">')
    else:
        if transparent:
            separator = widgets.HTML('<hr style="border: 2px solid transparent; margin: 20px 0;">')
        else:
            separator = widgets.HTML('<hr style="border: 2px solid #ccc; margin: 20px 0;">')
    display(separator)

def disable_scrolling():
    """Disabling scrolling of the output area"""
    display(widgets.HTML("""
<style>
  .output_scroll {  /* Class for scrollable outputs in classic Notebook */
     max-height: 10000px; /* Adjust max-height as needed */
     overflow-y: scroll;
  }
  .jp-OutputArea-output { /* Class for output areas in JupyterLab/Colab */
     max-height: 10000px;
     overflow-y: auto; /* or 'scroll' */
  }
</style>
"""))

# --- 7. Create Buttons and Text Widgets ---
disable_scrolling()  # Disable scrolling of the output area

text_label = widgets.Label(value=f'Target protein specification: ', layout=widgets.Layout(width='10%'))  # Label for the range specification text widget
text_widget = widgets.Text(value=target_protein_specification, disabled=True, layout=widgets.Layout(width='60%'))  # Text widget to display and hold the range specification
hotspot_text_label = widgets.Label(value=f'Hotspots: ', layout=widgets.Layout(width='10%')) # Label for the hotspot specification text widget
hotspot_text_widget = widgets.Text(value=hotspot_specification, disabled=True, layout=widgets.Layout(width='60%')) # Text widget to display and hold the hotspot specification
display(widgets.HBox([text_label,text_widget]))  # Display label and text widget for range specification horizontally
display(widgets.HBox([hotspot_text_label, hotspot_text_widget])) # Display label and text widget for hotspot specification horizontally

add_separator(transparent=True)

output_area = widgets.Output()  # Output area for displaying messages and errors
display(output_area)  # Display the output area widget

print ('\nCopy text from "Target protein specification" and "Hotspots" into the submission form')

add_separator()

add_range_button = widgets.Button(description="Add Target Selection")  # Button to add range selectors
add_range_button.on_click(add_range_selection_clicked)  # Attach callback

add_hotspot_button = widgets.Button(description="Add Hotspot")  # Button to add hotspot selectors
add_hotspot_button.on_click(add_hotspot_selection_clicked)  # Attach callback

collect_selections_button = widgets.Button(description="Display Specification")  # Button to update specification text
collect_selections_button.on_click(display_selections_clicked)  # Attach callback

remove_range_button = widgets.Button(description="Remove Target Selection")
remove_range_button.on_click(remove_range_selection_clicked)

remove_hotspot_button = widgets.Button(description="Remove Hotspot")
remove_hotspot_button.on_click(remove_hotspot_selection_clicked)


# --- 8. Display Widgets in Colab Output ---
display(widgets.HBox([add_range_button, add_hotspot_button]))  # Display add buttons horizontally
display(widgets.HBox([remove_range_button, remove_hotspot_button]))  # Display remove buttons horizontally
add_separator(transparent=True, thin=True)
print ('Use "Add Target Selection" button to specify more protein segments')
print ('Use "Add Hotspot" button to add more hotspot residues')
print ('Use "Remove Target Selection" and "Remove Hotspot" buttons to remove last dropdowns')
add_separator(transparent=True, thin=True)

# --- 9. Initialize Dropdowns on Load and Initial Specification Update ---
create_chain_residue_range_dropdowns()  # Create initial range selection dropdowns
add_separator(thin=True)
selection_widgets_hotspots = widgets.VBox()
display(selection_widgets_hotspots)
add_separator()

display(widgets.HBox([collect_selections_button]))
add_separator(transparent=True)
print ('Use "Display Specification" button to visualise target protein specification and hotspots in the molecular viewer\n')
mol_widget = widgets.Output()
display(mol_widget)
mol_viewer = display_structure(pdb_id, '', '', False, mol_widget)

print ('\nRegions that are not specified are displayed semi-transparently, while specified regions are solid')
print ('Hotspot residues are displayed as spheres\n\n')



# collect_selections_clicked('')  # Initialize the target protein specification text widget (empty initially)


In [ ]:
#@title 5. (Optional) Print target specification and hotspots

print(f'PDB id: {pdb_id}')
print(f'Target protein specification: {text_widget.value}')
print(f'Hotspots: {hotspot_text_widget.value}')
print(f'Binder size up to: {binder_length}')
